# EDA

In [1]:
import os
print(os.getcwd())

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


c:\Users\junio\OneDrive\바탕 화면\dacon_elec


In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
building_info = pd.read_csv('./data/building_info.csv')

In [3]:
train['month'] = train['일시'].apply(lambda x : int(x[4:6]))
train['day'] = train['일시'].apply(lambda x : int(x[6:8]))
train['time'] = train['일시'].apply(lambda x : int(x[9:11]))
test['month'] = test['일시'].apply(lambda x : int(x[4:6]))
test['day'] = test['일시'].apply(lambda x : int(x[6:8]))
test['time'] = test['일시'].apply(lambda x : int(x[9:11]))

In [4]:
train.drop(columns=['일시','num_date_time'], axis=1, inplace=True)
test.drop(columns=['일시','num_date_time'], axis=1, inplace=True)

In [5]:
col={'건물번호':'building_number',
                            '일시':'date_time',
                            '기온(C)':'temperature',
                            '강수량(mm)':'rainfall',
                            '풍속(m/s)':'windspeed',
                            '습도(%)':'humidity',
                            '일조(hr)':'sunshine',
                            '일사(MJ/m2)':'solar_radiation',
                            '전력소비량(kWh)':'power_consumption'}
train=train.rename(columns=col)
test=test.rename(columns=col)


In [6]:
building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}
building_info['building_type'] = building_info['building_type'].replace(translation_dict)
print(building_info.building_type.unique())

['Other Buildings' 'Public' 'University' 'Data Center'
 'Department Store and Outlet' 'Hospital' 'Commercial' 'Apartment'
 'Research Institute' 'Knowledge Industry Center' 'Discount Mart'
 'Hotel and Resort']


In [7]:
print('Train column : ','\n',train.columns)
print('Test column : ','\n',test.columns)
print(building_info.columns)
print("--------------")

print(train.shape)
print(test.shape)
print(building_info.shape)

Train column :  
 Index(['building_number', 'temperature', 'rainfall', 'windspeed', 'humidity',
       'sunshine', 'solar_radiation', 'power_consumption', 'month', 'day',
       'time'],
      dtype='object')
Test column :  
 Index(['building_number', 'temperature', 'rainfall', 'windspeed', 'humidity',
       'month', 'day', 'time'],
      dtype='object')
Index(['building_number', 'building_type', 'total_area', 'cooling_area',
       'solar_power_capacity', 'ess_capacity', 'pcs_capacity'],
      dtype='object')
--------------
(204000, 11)
(16800, 8)
(100, 7)


In [8]:
train_m=pd.merge(train,building_info, on="building_number",how="left")
test_m=pd.merge(test,building_info, on="building_number",how="left")

In [10]:
train_m[['building_number','building_type','month','day','time','power_consumption']]

,building_number,building_type,month,day,time,power_consumption
0,1,Other Buildings,6,1,0,1085.28
1,1,Other Buildings,6,1,1,1047.36
2,1,Other Buildings,6,1,2,974.88
3,1,Other Buildings,6,1,3,953.76
4,1,Other Buildings,6,1,4,986.40
...,...,...,...,...,...,...
203995,100,Hotel and Resort,8,24,19,881.04
203996,100,Hotel and Resort,8,24,20,798.96
203997,100,Hotel and Resort,8,24,21,825.12
203998,100,Hotel and Resort,8,24,22,640.08


In [11]:
print(train_m.shape)
print(test_m.shape)

(204000, 17)
(16800, 14)


# 결측치 처리

In [13]:
train_m

,building_number,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,month,day,time,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity
0,1,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,6,1,0,Other Buildings,110634.00,39570.00,-,-,-
1,1,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,6,1,1,Other Buildings,110634.00,39570.00,-,-,-
2,1,17.7,NaN,1.5,45.0,NaN,NaN,974.88,6,1,2,Other Buildings,110634.00,39570.00,-,-,-
3,1,16.7,NaN,1.4,48.0,NaN,NaN,953.76,6,1,3,Other Buildings,110634.00,39570.00,-,-,-
4,1,18.4,NaN,2.8,43.0,NaN,NaN,986.40,6,1,4,Other Buildings,110634.00,39570.00,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,NaN,0.9,86.0,0.5,NaN,881.04,8,24,19,Hotel and Resort,57497.84,40035.23,-,-,-
203996,100,22.4,NaN,1.3,86.0,0.0,NaN,798.96,8,24,20,Hotel and Resort,57497.84,40035.23,-,-,-
203997,100,21.3,NaN,1.0,92.0,NaN,NaN,825.12,8,24,21,Hotel and Resort,57497.84,40035.23,-,-,-
203998,100,21.0,NaN,0.3,94.0,NaN,NaN,640.08,8,24,22,Hotel and Resort,57497.84,40035.23,-,-,-


In [14]:
train_m.isna().sum()

building_number              0
temperature                  0
rainfall                160069
windspeed                   19
humidity                     9
sunshine                 75182
solar_radiation          87913
power_consumption            0
month                        0
day                          0
time                         0
building_type                0
total_area                   0
cooling_area                 0
solar_power_capacity         0
ess_capacity                 0
pcs_capacity                 0
dtype: int64

In [16]:
test_m.isna().sum()

building_number         0
temperature             0
rainfall                0
windspeed               0
humidity                0
month                   0
day                     0
time                    0
building_type           0
total_area              0
cooling_area            0
solar_power_capacity    0
ess_capacity            0
pcs_capacity            0
dtype: int64

In [12]:
print(len(train_m[train_m['solar_power_capacity']=='-']))
print(len(train_m[train_m['ess_capacity'] == '-']))
print(len(train_m[train_m['pcs_capacity'] == '-']))

130560
193800
193800


결측치 있는 columns : 

- rainfall : test data에 존재하는 column이라 결측치 대체해야함 

- sunshine, solar_radiation : test data에는 존재하지 않음

- ess_capacity, pcs_capacity, solar_power_capacity : "-"로 결측치 


결측치 비율 살펴보자

In [23]:
print("아래는 train set 내 column의 결측치 비율이다.")
print("\n")
print("rainfall : ",round(train_m.rainfall.isna().sum()/len(train_m)*100,2),"%")
print("sunshine : ",round(train_m.sunshine.isna().sum()/len(train_m)*100,2),"%")
print("solar_radiation : ",round(train_m.solar_radiation.isna().sum()/len(train_m)*100,2),"%")
print("-----------")
print("solar_power_capacity : ",round(len(train_m[train_m['solar_power_capacity']=='-'])/len(train_m)*100,2),"%")
print("ess_capacity : ",round(len(train_m[train_m['ess_capacity'] == '-'])/len(train_m)*100,2),"%")
print("pcs_capacity : ",round(len(train_m[train_m['pcs_capacity'] == '-'])/len(train_m)*100,2),"%")

아래는 train set 내 column의 결측치 비율이다.


rainfall :  78.47 %
sunshine :  36.85 %
solar_radiation :  43.09 %
-----------
solar_power_capacity :  64.0 %
ess_capacity :  95.0 %
pcs_capacity :  95.0 %


IDEA : 결측치를 어떻게 handling할 것인지에 따라서 성능 차이가 많이 날듯

IDEA : group_by 할 수 있는 factor가 너무 많다. factor들을 정리해보고, factor간 관계를 파악하는게 중요하게 작용할 듯

IDEA : time과 상관없는 cross-sectional data : building_info 존재. 시계열 예측에 적용하려면, 적어도 시계열 자료를 cross-sectional 하게 만든 data로써 사용해야 할 것임. 

(해당 data는 `bulding_number`에 따른 데이터임. `total_area`, `cooling_area` 변수가 시계열 자료에 어떤 영향을 미치는 지 파악할 것)

In [34]:
train_solar_power = train_m.drop(train_m[train_m['solar_power_capacity']=='-'].index)
train_ess = train_m.drop(train_m[train_m['ess_capacity']=='-'].index)
train_pcs = train_m.drop(train_m[train_m['pcs_capacity']=='-'].index)

print(len(train_solar_power))
print(len(train_ess))
print(len(train_pcs))

train_rainfall = train_m.drop(train_m[train_m['rainfall'].isna()==True].index)
train_sunshine = train_m.drop(train_m[train_m['sunshine'].isna()==True].index)
train_solar_radiation = train_m.drop(train_m[train_m['solar_radiation'].isna()==True].index)

print(len(train_rainfall))
print(len(train_sunshine))
print(len(train_solar_radiation))

73440
10200
10200
43931
128818
116087


In [68]:
print(train_rainfall.rainfall.describe())

count    43931.000000
mean         2.040830
std          5.165218
min          0.000000
25%          0.000000
50%          0.100000
75%          1.300000
max         92.200000
Name: rainfall, dtype: float64


In [66]:
train_rainfall.head()

,building_number,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,month,day,time,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity
104,1,22.9,0.0,1.3,55.0,0.0,0.25,1440.48,6,5,8,Other Buildings,110634.0,39570.0,-,-,-
105,1,23.6,0.0,1.2,53.0,0.0,0.51,1814.40,6,5,9,Other Buildings,110634.0,39570.0,-,-,-
116,1,20.9,0.0,3.2,64.0,0.0,0.02,2054.40,6,5,20,Other Buildings,110634.0,39570.0,-,-,-
117,1,20.5,0.0,2.4,70.0,NaN,NaN,1716.00,6,5,21,Other Buildings,110634.0,39570.0,-,-,-
123,1,19.3,1.3,1.5,84.0,NaN,NaN,964.32,6,6,3,Other Buildings,110634.0,39570.0,-,-,-


# factors

- building_type : 건물 유형 (14 levels), categorical

- day : 요일에 따라 (time) # 주말에는 일을 하지 않으니 y 감소 # 그러나 이것도 직장/여가공간 으로 분류하면 다르게 나올수도 있을것 같음

- time : 시간대에 따라 (time) # 밤에는 일을 하지 않으니 y 감소

- temperature : 온도 (시계열 간 관계성 파악 method 사용할 것)

- total area -> decompose했을때의 mean power consumption 예측 (group_by : building_number)

- cooling area -> decompose했을때의 mean power consumption 예측 (group_by : building_number)

- ESS, PCS : 유무/ 저장했다가 전력난때 사용하는 장치 -> decompose했을때의 mean power consumption 예측 (group_by : building_number)

- rainfall, windspeed, humidity, sunshine, solar_radiation : DTW, Time Lagged Cross Correlation?, CoIntegration, Granger Causality, VAR, VECM, bsts 사용해볼것

IDEA : 얼마나 Categorize를 잘하는지가 관건이 될거 같음

### etc

기존 probabilistic graphical model : y~x, x~y 돌려보고 더 설명이 잘 되는 것들로 인과관계 정립

probabilistic graphical model for time series data? : SNMM? (group_by building 해서 mean으로?)

**cross-sectional data만들때 group_by=building_number 별 power consumption의 mean과 variance, velocity, velocity variance 다 학습할 것임**

categorize가 자세하게 이루어지면, 결국은 단기 시계열 예측의 문제일수도 있음. 단기 시계열 예측 성능이 좋은 모델을 사용해봐도 좋을 것 같다.